In [1]:
from pycocotools.coco import COCO

In [2]:
%pylab --no-import-all
%reload_ext autoreload
%autoreload 2
#%matplotlib inline

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
import json
from pathlib import Path
from pprint import pprint

In [4]:
import pandas as pd
import numpy as np

In [6]:
json_dir = Path('./data/training/GT')

In [7]:
data = []

In [8]:
for file in json_dir.iterdir():
    if str(file).endswith('json'):
        fdata = json.load(file.open())
        data.extend(fdata)

In [9]:
data[0]

{u'annotations': [{u'class': u'Person',
   u'height': 161.0,
   u'width': 145.0,
   u'x': 79.0,
   u'y': 164.0},
  {u'class': u'Person',
   u'height': 204.0,
   u'width': 152.0,
   u'x': 280.0,
   u'y': 148.0},
  {u'class': u'Left Shoulder',
   u'x': 183.59237462904284,
   u'y': 234.89764096373426},
  {u'class': u'Head', u'x': 140.8379860168, u'y': 271.11312308234},
  {u'class': u'Head', u'x': 391.8266319533878, u'y': 256.275797115459},
  {u'class': u'Right Shoulder',
   u'x': 340.1478761714274,
   u'y': 228.3184374301362},
  {u'class': u'Right Elbow', u'x': 315.1556909981843, u'y': 251.6162371679052},
  {u'class': u'Right Hand',
   u'x': 329.13437084084575,
   u'y': 304.14218566760263},
  {u'class': u'Left Elbow',
   u'x': 205.44423405123575,
   u'y': 237.21396096637528},
  {u'class': u'Left Hand', u'x': 177.0632780070444, u'y': 262.2061461396184},
  {u'class': u'Right Shoulder',
   u'x': 121.24598579893858,
   u'y': 239.50561726785403},
  {u'class': u'Right Elbow',
   u'x': 121.84325

In [10]:
pdata = pd.DataFrame(data)

In [11]:
pdata[:3]

,annotations,class,filename
0,"[{u'y': 164.0, u'width': 145.0, u'x': 79.0, u'...",image,data/training/10_16_T1_K1/frame00325.jpg
1,"[{u'y': 165.0, u'width': 159.0, u'x': 74.0, u'...",image,data/training/10_16_T1_K1/frame00431.jpg
2,"[{u'y': 161.0, u'width': 151.0, u'x': 78.0, u'...",image,data/training/10_16_T1_K1/frame00636.jpg


### Create coco-style data

In [12]:
coco_data = {"images":[], "annotations":[], "categories":[]}

In [13]:
coco_data["categories"] = [{'supercategory': 'person',
  'id': 1,
  'name': 'person',
  'keypoints': ['nose',
   'left_eye',
   'right_eye',
   'left_ear',
   'right_ear',
   'left_shoulder',
   'right_shoulder',
   'left_elbow',
   'right_elbow',
   'left_wrist',
   'right_wrist',
   'left_hip',
   'right_hip',
   'left_knee',
   'right_knee',
   'left_ankle',
   'right_ankle'],
  'skeleton': [[16, 14],
   [14, 12],
   [17, 15],
   [15, 13],
   [12, 13],
   [6, 12],
   [7, 13],
   [6, 7],
   [6, 8],
   [7, 9],
   [8, 10],
   [9, 11],
   [2, 3],
   [1, 2],
   [1, 3],
   [2, 4],
   [3, 5],
   [4, 6],
   [5, 7]]}]

#### Our Annotation format

In our annotations, we only have head(nose)(0), left shoulder(5), right shoulder(6), left elbow(7), right elbow(8), left wrist(9), right wrist(10), bb, arms crossed, both arms on table,
both arms on body, one arm on body, one arm on table, leaning forward, leaning backward, upright

In [14]:
joint_ann_dict = {"Head":0, "Left Shoulder":5, "Right Shoulder":6, "Left Elbow":7, "Right Elbow":8, "Left Hand":9, "Right Hand": 10}

In [15]:
#h, w = 424, 512

In [16]:
import cv2, os

In [17]:
i = 0
i_ann = 0

for i, item in pdata.iterrows():
    #item = pdata.iloc[i]
    #pprint(item)
    fname = item['filename']
    fname = fname.replace("Kinect","depth")
    fname = fname.replace("ir","ud")
    h, w, _ = cv2.imread(fname).shape
    im = {"height":h, "width":w, "file_name":fname, "id":i}

    coco_data["images"].append(im)

    annpd = pd.DataFrame(item['annotations'])
    
    eps = 15
    for _, person in annpd[annpd["class"]=="Person"].iterrows():
        #print(person)
        ann = {"segmentation":[], "keypoints":[], "iscrowd":0, "num_keypoints":17, "area":0, "id":None, "image_id":i, "bbox":[], "category_id":1}
        ann["bbox"] = [person["x"], person["y"], person["width"], person["height"]]
        ann["area"] = person["width"] * person["height"]
        condition = (annpd["x"] > person["x"]-eps) & (annpd["x"] < person["x"]+person["width"]+eps) & \
                    (annpd["y"] > person["y"]-eps) & (annpd["y"] < person["y"]+person["height"]+eps)
        
        joints = annpd[condition & (annpd["class"]!="Person")]

        kp = np.zeros(51, dtype=np.int)
        for _, row in joints.iterrows():
            if row["class"] in joint_ann_dict:
                idx = 3 * joint_ann_dict[row["class"]]
                kp[idx:idx+3] = int(row["x"]), int(row["y"]), 2
        ann["keypoints"] = kp.tolist()
        ann["id"] = i_ann
        ann["image_id"] = i

        coco_data["annotations"].append(ann)

        i_ann += 1
        #print(kp)

        #print(joints)

In [18]:
coco_data['annotations'][:2]

[{'area': 23345.0,
  'bbox': [79.0, 164.0, 145.0, 161.0],
  'category_id': 1,
  'id': 0,
  'image_id': 0,
  'iscrowd': 0,
  'keypoints': [140,
   271,
   2,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   183,
   234,
   2,
   121,
   239,
   2,
   205,
   237,
   2,
   121,
   261,
   2,
   177,
   262,
   2,
   130,
   292,
   2,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'num_keypoints': 17,
  'segmentation': []},
 {'area': 31008.0,
  'bbox': [280.0, 148.0, 152.0, 204.0],
  'category_id': 1,
  'id': 1,
  'image_id': 0,
  'iscrowd': 0,
  'keypoints': [391,
   256,
   2,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   415,
   233,
   2,
   340,
   228,
   2,
   0,
   0,
   0,
   315,
   251,
   2,
   381,
   314,
   2,
   329,
   304,
   2,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'num_keypoints'

### Write annotations

In [19]:
annfile = Path('annotations/keypoints.json')

In [20]:
json.dump(coco_data, open(str(annfile), mode='w'))

### Test if annotation is correct

In [21]:
from pycocotools.coco import COCO
import numpy as np
#import skimage.io as io
import matplotlib.pyplot as plt

In [22]:
import cv2

In [23]:
annfile = Path('annotations/keypoints.json')

In [24]:
coco = COCO(str(annfile))

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [25]:
imgIds = coco.getImgIds(catIds=[1])
imgs = coco.loadImgs(ids = imgIds)

In [26]:
dim = [2, 2]
for i,img in enumerate(imgs):
    fname = img['file_name']
    #fname = fname.replace("Kinect","depth")
    #fname = fname.replace("ir","ud")
    print(fname)
    I = cv2.imread(fname)
    annIds = coco.getAnnIds(imgIds=img['id'], iscrowd=None)
    anns = coco.loadAnns(annIds)
    for ann in anns:
        kp = ann['keypoints']
        for j in range(17):
            if kp[3*j+2]==2:
                x, y = kp[3*j], kp[3*j+1]
                cv2.circle(I,(x, y), 5, (0, 255, 0), 3)
        bb = [int(f) for f in ann['bbox']]
        cv2.rectangle(I, (bb[0], bb[1]), (bb[0]+bb[2], bb[1]+bb[3]), (255, 0, 0), 3)
    ff = Path(fname).name
    cv2.imwrite('checkim/{}.jpg'.format(ff), I)

data/training/10_16_T1_K1/frame00325.jpg
data/training/10_16_T1_K1/frame00431.jpg
data/training/10_16_T1_K1/frame00636.jpg
data/training/10_16_T1_K1/frame00891.jpg
data/training/10_16_T1_K1/frame00940.jpg
data/training/10_16_T1_K1/frame01070.jpg
data/training/10_16_T1_K1/frame01232.jpg
data/training/10_16_T1_K1/frame01241.jpg
data/training/10_16_T1_K1/frame01278.jpg
data/training/10_16_T1_K1/frame01288.jpg
data/training/10_16_T1_K1/frame01324.jpg
data/training/10_16_T1_K1/frame01348.jpg
data/training/10_16_T1_K1/frame01739.jpg
data/training/10_16_T1_K1/frame01888.jpg
data/training/10_16_T1_K1/frame02208.jpg
data/training/10_16_T1_K1/frame02287.jpg
data/training/10_16_T1_K1/frame02362.jpg
data/training/10_16_T1_K1/frame02529.jpg
data/training/10_16_T1_K1/frame02541.jpg
data/training/10_16_T1_K1/frame02704.jpg
data/training/10_16_T1_K1/frame02819.jpg
data/training/10_16_T1_K1/frame02835.jpg
data/training/10_16_T1_K1/frame02853.jpg
data/training/10_16_T1_K1/frame02937.jpg
data/training/10

## Split train and val